# STEPDISC with LDA model

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [2]:
# Chargement de la base
import pandas as pd
DTrain = pd.read_excel("./data/Data_Illustration_Livre_ADL.xlsx",sheet_name="DATA_2_TRAIN",header=0)
DTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TYPE    52 non-null     object 
 1   MEOH    52 non-null     float64
 2   ACET    52 non-null     float64
 3   BU1     52 non-null     float64
 4   BU2     52 non-null     float64
 5   ISOP    52 non-null     int64  
 6   MEPR    52 non-null     float64
 7   PRO1    52 non-null     float64
 8   ACAL    52 non-null     float64
dtypes: float64(7), int64(1), object(1)
memory usage: 3.8+ KB


In [3]:
DTest = pd.read_excel("./data/Data_Illustration_Livre_ADL.xlsx",sheet_name="DATA_2_TEST",header=0)
DTest.head()

,TYPE,MEOH,ACET,BU1,BU2,ISOP,MEPR,PRO1,ACAL
0,KIRSCH,3,15,0.2,30.0,9,9,350,9.0
1,KIRSCH,475,172,1.9,7.0,113,33,546,14.0
2,KIRSCH,186,101,0.0,1.6,36,11,128,8.0
3,KIRSCH,371,414,1.2,0.0,97,39,502,9.0
4,KIRSCH,583,226,2.3,19.0,120,46,656,11.0


In [4]:
from discrimintools import LDA

lda = LDA(target=["TYPE"],priors="prop")

# Instanciation
lda.fit(DTrain)

LDA(priors='prop', target=['TYPE'])

## Forward model

### Selection

In [5]:
from discrimintools import STEPDISC
forward = STEPDISC(model=lda,method="forward",alpha=0.01,verbose=True,model_train=True)

      Wilks L.  Partial L.          F       p-value
MEOH  0.282629    0.717371  62.186129  3.586020e-14
ACET  0.971855    0.028145   0.709531  2.540218e-01
BU1   0.286173    0.713827  61.112585  4.873879e-14
BU2   0.914588    0.085412   2.288014  1.122087e-01
ISOP  0.887731    0.112269   3.098457  5.406192e-02
MEPR  0.691854    0.308146  10.912106  1.203236e-04
PRO1  0.835465    0.164535   4.824978  1.222491e-02
ACAL  0.979642    0.020358   0.509127  1.511647e-01

      Wilks L.  Partial L.          F   p-value
ACET  0.253614    0.102660   2.745708  0.074297
BU1   0.192547    0.318729  11.228252  0.000100
BU2   0.244101    0.136320   3.788072  0.029680
ISOP  0.264061    0.065697   1.687604  0.195751
MEPR  0.221217    0.217287   6.662572  0.002796
PRO1  0.255676    0.095365   2.530037  0.090232
ACAL  0.235697    0.166054   4.778852  0.012803

      Wilks L.  Partial L.         F   p-value
ACET  0.178725    0.071786  1.817445  0.173671
BU2   0.170291    0.115585  3.071236  0.055772
ISOP 

### Selected variables

In [6]:
forward.results_["selected"]

['MEOH', 'BU1', 'MEPR']

### Selected model

#### Coefficients

In [7]:
forward.results_["train"].coef_

,KIRSCH,MIRAB,POIRE
MEOH,0.006922,0.021321,0.022619
BU1,-0.076617,0.401044,0.373522
MEPR,0.086678,-0.032474,0.046747


In [8]:
forward.results_["train"].intercept_

,KIRSCH,MIRAB,POIRE
Intercept,-3.610717,-14.775375,-18.371099


#### Statistical evaluation

In [9]:
forward.results_["train"].statistics_["statistical_evaluation"]

,Wilks L.,Partial L.,"F(2, 47)",p-value
MEOH,0.220542,0.670102,11.569265,0.000082
BU1,0.221217,0.668058,11.676585,0.000076
MEPR,0.192547,0.767532,7.117602,0.001994


## Backward selection

In [10]:
from discrimintools import STEPDISC
backward = STEPDISC(model=lda,method="backward",alpha=0.01,verbose=True,model_train=True)

      Wilks L.  Partial L.          F   p-value
MEOH  0.117975    0.434512  16.136067  0.000006
ACET  0.074153    0.100333   2.341965  0.108572
BU1   0.084183    0.207525   5.499262  0.007563
BU2   0.095695    0.302858   9.122996  0.000513
ISOP  0.072310    0.077400   1.761764  0.184196
MEPR  0.087798    0.240148   6.636945  0.003128
PRO1  0.092396    0.277962   8.084336  0.001071
ACAL  0.075884    0.120850   2.886700  0.066885

      Wilks L.  Partial L.          F   p-value
MEOH  0.129692    0.442449  17.061483  0.000004
ACET  0.079826    0.094151   2.234643  0.119316
BU1   0.092945    0.222011   6.135344  0.004528
BU2   0.109809    0.341492  11.149568  0.000126
MEPR  0.098171    0.263427   7.689228  0.001397
PRO1  0.101216    0.285585   8.594537  0.000724
ACAL  0.081813    0.116149   2.825379  0.070331

      Wilks L.  Partial L.          F   p-value
MEOH  0.147097    0.457327  18.540063  0.000001
BU1   0.098601    0.190415   5.174419  0.009589
BU2   0.122087    0.346156  11.647178 

### Train model

In [11]:
train_model = backward.results_["train"]

In [12]:
train_model.coef_

,KIRSCH,MIRAB,POIRE
MEOH,0.006172,0.026956,0.031912
BU1,-0.086181,0.346433,0.280117
BU2,-0.005208,0.070420,0.116972
MEPR,0.086537,-0.025564,0.058299
PRO1,0.002536,-0.005289,-0.008360


In [13]:
train_model.intercept_

,KIRSCH,MIRAB,POIRE
Intercept,-4.411341,-16.918659,-24.263717


In [14]:
train_model.statistics_["statistical_evaluation"]

,Wilks L.,Partial L.,"F(2, 45)",p-value
MEOH,0.155113,0.557582,17.852822,0.000002
BU1,0.110174,0.785016,6.161841,0.004313
BU2,0.136572,0.633277,13.029454,0.000034
MEPR,0.111753,0.773921,6.572738,0.003131
PRO1,0.131479,0.657808,11.704517,0.000081


In [15]:
train_model.predict_proba(DTest).head(6)

,KIRSCH,MIRAB,POIRE
0,0.999999,9.787139e-07,1.857075e-09
1,0.999902,9.560699e-05,2.362601e-06
2,0.999979,2.133381e-05,6.335311e-08
3,0.999996,3.449756e-06,7.289128e-08
4,0.999698,2.617153e-04,3.990588e-05
5,1.000000,4.987397e-07,3.530405e-10


In [16]:
train_model.predict(DTest).head(6)

0    KIRSCH
1    KIRSCH
2    KIRSCH
3    KIRSCH
4    KIRSCH
5    KIRSCH
Name: prediction, dtype: object

In [17]:
train_model.pred_table()

prediction,KIRSCH,MIRAB,POIRE
TYPE,,,
KIRSCH,17,0,0
MIRAB,0,13,2
POIRE,0,1,19


## Stepwise with large Dataset

In [18]:
# chargement des données
DTrainLarge = pd.read_excel("./data/Data_Illustration_Livre_ADL.xlsx",sheet_name="WAVE_NOISE", engine="openpyxl")
DTrainLarge.head(6)

,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,...,noisy32,noisy33,noisy34,noisy35,noisy36,noisy37,noisy38,noisy39,noisy40,classe
0,-1.23,-1.56,-1.75,-0.28,0.60,2.22,0.85,0.21,-0.20,0.89,...,1.61,0.20,0.28,2.90,1.85,-0.75,6.06,-1.02,-0.53,C
1,-0.69,2.43,0.61,2.08,2.30,3.25,5.52,4.55,2.97,2.22,...,3.24,1.92,2.29,2.13,-0.40,-0.93,-0.14,0.50,-0.94,B
2,-0.12,-0.94,1.29,2.59,2.42,3.55,4.94,3.25,1.90,2.07,...,0.06,-1.47,-0.26,-0.24,3.70,1.89,0.52,-1.61,0.05,A
3,0.86,0.29,2.19,-0.02,1.13,2.51,2.37,5.45,5.45,4.84,...,-0.54,-2.78,-3.72,-2.76,2.94,-2.71,-3.29,-0.41,-0.45,B
4,1.16,0.37,0.40,-0.59,2.66,1.00,2.69,4.06,5.34,3.53,...,2.16,4.17,-0.84,-5.35,-3.63,-4.70,2.71,-2.86,-1.39,B
5,0.00,0.77,1.32,0.29,-1.28,0.84,1.60,1.55,2.93,4.76,...,6.03,-0.96,-0.94,-1.03,-3.43,-0.56,-3.05,-0.33,5.34,C


In [19]:
lda2 = LDA(target=["classe"],priors="prop").fit(DTrainLarge)

In [20]:
forward2 = STEPDISC(model=lda2,method="forward",alpha=0.01,verbose=True,model_train=True)

         Wilks L.  Partial L.            F       p-value
V01      0.999400    0.000600     1.499319  2.233827e-01
V02      0.947882    0.052118   137.375321  1.110223e-16
V03      0.853547    0.146453   428.697599  1.110223e-16
V04      0.750799    0.249201   829.286754  1.110223e-16
V05      0.668073    0.331927  1241.358322  1.110223e-16
...           ...         ...          ...           ...
noisy36  0.999648    0.000352     0.879410  3.208226e-01
noisy37  0.999953    0.000047     0.116597  1.912612e-04
noisy38  0.999103    0.000897     2.243096  1.062363e-01
noisy39  0.999906    0.000094     0.234189  1.403216e-02
noisy40  0.999725    0.000275     0.687115  2.334154e-01

[61 rows x 4 columns]

         Wilks L.  Partial L.            F       p-value
V01      0.619365    0.000554     1.384327  2.505883e-01
V02      0.612597    0.011475    28.996684  3.016476e-13
V03      0.597780    0.035385    91.633051  1.110223e-16
V04      0.574247    0.073360   197.760422  1.110223e-16
V05    

In [21]:
forward2.results_["selected"]

['V07',
 'V11',
 'V17',
 'V06',
 'V10',
 'V12',
 'V08',
 'V13',
 'V09',
 'V05',
 'V18',
 'V04',
 'V15',
 'V14',
 'V16',
 'V03']

In [22]:
train_model2 = forward2.results_["train"]
train_model2.coef_

,A,B,C
V07,2.872171,2.842879,2.594084
V11,1.573334,2.024593,1.972344
V17,1.970818,1.645235,1.780870
V06,2.508800,2.434391,2.192368
V10,1.938602,2.348692,2.274755
V12,2.119623,2.390649,2.491738
V08,2.446321,2.651350,2.361896
V13,2.017536,2.180367,2.356992
V09,1.890088,2.220659,2.034742
V05,2.260993,2.177873,1.967679


In [23]:
train_model2.intercept_

,A,B,C
Intercept,-39.210316,-42.634605,-41.671901


In [24]:
backward2 = STEPDISC(model=lda2,method="backward",alpha=0.01,verbose=True,model_train=True)

         Wilks L.  Partial L.          F       p-value
V01      0.289014    0.000202   0.498011  1.343682e-01
V02      0.289116    0.000554   1.368362  2.546200e-01
V03      0.289562    0.002095   5.181460  5.650395e-03
V04      0.290403    0.004985  12.367286  4.388999e-06
V05      0.291657    0.009262  23.077602  1.057022e-10
...           ...         ...        ...           ...
noisy36  0.288964    0.000027   0.066634  3.177468e-07
noisy37  0.288990    0.000119   0.294420  3.356845e-02
noisy38  0.289253    0.001028   2.539364  7.901963e-02
noisy39  0.288993    0.000131   0.322185  4.496568e-02
noisy40  0.288997    0.000144   0.356461  6.058185e-02

[61 rows x 4 columns]



In [25]:
backward2.results_["selected"]

['V01',
 'V02',
 'V03',
 'V04',
 'V05',
 'V06',
 'V07',
 'V08',
 'V09',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'noisy1',
 'noisy2',
 'noisy3',
 'noisy4',
 'noisy5',
 'noisy6',
 'noisy7',
 'noisy8',
 'noisy9',
 'noisy10',
 'noisy11',
 'noisy12',
 'noisy13',
 'noisy14',
 'noisy15',
 'noisy16',
 'noisy17',
 'noisy18',
 'noisy19',
 'noisy20',
 'noisy21',
 'noisy22',
 'noisy23',
 'noisy24',
 'noisy25',
 'noisy26',
 'noisy27',
 'noisy28',
 'noisy29',
 'noisy30',
 'noisy31',
 'noisy32',
 'noisy33',
 'noisy34',
 'noisy35',
 'noisy36',
 'noisy37',
 'noisy38',
 'noisy39',
 'noisy40']

In [26]:
backward2.results_["train"].coef_

,A,B,C
V01,-0.105525,-0.069218,-0.062112
V02,0.342431,0.312313,0.264070
V03,1.089313,1.004553,0.939360
V04,1.720041,1.679991,1.505094
V05,2.304870,2.222977,2.005614
...,...,...,...
noisy36,-0.060001,-0.056932,-0.063803
noisy37,0.033223,0.037119,0.047017
noisy38,0.027901,-0.004162,0.032963
noisy39,0.014570,0.001613,0.013632
